# Synopsis

Extracting information from scanned files.

# Words to remember

**warping**

**denoising**


# Read libraries

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from colorama import Back, Fore, Style
from copy import copy, deepcopy
from pathlib import Path


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np

from matplotlib.gridspec import GridSpec
from matplotlib.patches import Circle
from pylab import imread, imshow, imsave
from scipy.stats import pearsonr
from skimage import img_as_float, img_as_ubyte
from skimage.color import rgb2gray
from skimage.filters import rank, threshold_otsu, gaussian
from skimage.measure import find_contours
from skimage.morphology import ( disk, binary_dilation, binary_erosion, 
                                 binary_closing, binary_opening, 
                                 remove_small_holes, remove_small_objects,
                                 flood_fill, )
from skimage.util import random_noise

from skimage.transform import estimate_transform, warp

from module_libraries.my_stats import half_frame
from module_libraries.image_lib import display_all_channels, grayscale_zoom

my_fontsize = 15
data_folder = Path.cwd() / 'Data' / 'Scanned_Images'
results_folder = Path.cwd() / 'Results'


# Load images

We load all images but select a single one for further analysis.


In [ ]:
my_images = list( data_folder.glob('*.png') )
print( Style.BRIGHT, f"There are {len(my_images)} images in the folder.\n",
       Style.RESET_ALL )

for i in range(len(my_images)):
    print('\t', my_images[i].parts[-1])


In [ ]:
i = 3
plate = imread( my_images[i] )
print(f"That fourth channel is mostly ones:\n{plate[:10, :10, 3]}")

plate = np.uint8( 255 * plate )
print(f"Image '{i}' has shape {plate.shape}.\n")

imshow(plate);

We do not need the fourth channel, so we will get rid of it.

We will also want to work with a grayscale version of the image.  The question is: 

> **Which grayscale version should we use?**

Let's look at each channel separately besides a conversion to grayscale of the color image...

In [ ]:
display_all_channels( plate )

Not surprisingly, as the bars are green, the **green channel** seems to be the one where the text information and the boxes with data we want to extract is more clearly visible.

From now on, we will focus on this channel.

In [ ]:
imshow(plate[:,:,1], cmap = 'gray' );
plate[:,:,1]

In [ ]:
# Will call it plate_b for best
#
plate_b = plate[:,:,1]

print( f"Maximum of green channel is {plate_b.max()}, "
       f" minimum is {plate_b.min()}\n")

fig = plt.figure( figsize = (12, 10) )
plt.imshow( plate_b, cmap = 'gray' );

# Correct image perspective

This involves two steps.  First, we will get the coordinates of the 4 corners of the blue screen as accurately as possible.  To this end, we will magnify the region around each corner one at a time, and adjust the center of the zoomed in region until the red dot is located precisely at the corner.

Next, we use the `transform` package to correct the perspective of the image.  To this end, we need to provide new coordinates for the corners of the blue screen.

## Specify coordinates of corners of blue screen

We will use a gray scale version of the image since the zoom in function only operates with gray scale images.  
 

In [ ]:
# For distorted image, I know that corners are at: 
#    [[0,0], [900, 30], [950, 460], [40, 400]]

points_interest = [[0,0], [900, 30], [950, 460], [40, 400]]

#If we did not know, then we would start with empty list
#points_interest = [[], [], [], []]

print(points_interest)

**If we do not know the location of the points of interest (corners)**, then you can uncomment the code in next cell and run it until you get all sets of coordinates.

**Change the value of `k` when determining the coordinates of the points of interest with index `k`.** 

In [ ]:
# fig = plt.figure( figsize = (10, 6))
# ax = fig.add_subplot(111)

# zoom_factor = 8
# k = 2
# x = 3546
# y = 2788
# zoomed_image, x0, y0 = grayscale_zoom(plate_b, x, y, zoom_factor)


# ax.imshow( zoomed_image, cmap = 'gray', vmin = 0, vmax = 255 )
# ax.plot([zoom_factor*(x-x0)], [lzoom_factor*(y-y0)], 'ro');

# # Update coordinates of corner k
# #
# points_interest[k] = [x, y]
# print(points_interest)

## Correct perspective

We specify the desired coordinates for the corners of the blue screen in such a way that its size and location are approximately preserved.

In order to accomplish this, we **maintain the coordinates of the first corner** and pick the **coordinates of the opposite corner using the largest values of the coordinates from the other corners**.

We then use the original and desired corner coordinates to define a matrix transformation using `transform.estimate_transform`.

Finally, use apply `transform.warp` to correct the perspective of the image. 


In [ ]:
need_for_warp_correction = False

if not need_for_warp_correction:
    plate_warp = plate_b
    color_plate_warp = plate[:,:,:3]

else:
    print(points_interest)
    transformed_points = [[0,0], [1000, 0], [1000, 450], [0, 450]]
    print(transformed_points)
    
    tform = estimate_transform( 'projective', np.array(points_interest), 
                                np.array(transformed_points) )
    
    
    plate_warp = (255 * warp(plate_b, tform.inverse)).astype( np.uint8 )
    color_plate_warp = warp(plate[:,:,:3], tform.inverse)    

In [ ]:
fig = plt.figure(figsize = (12, 12))

ax1 = fig.add_subplot(121)
ax1.imshow(plate)

for point in points_interest:
    ax1.add_patch(Circle(point, 10, facecolor = 'r'))

ax2 = fig.add_subplot(122)
ax2.imshow( plate_warp, cmap = 'gray', vmin = 0, vmax = 255 )
for point in transformed_points:
    ax2.add_patch(Circle(point, 10, facecolor = 'r'));

**Looking good!!!**

In [ ]:
plate_corrected = plate_warp
color_plate_corrected = color_plate_warp

fig = plt.figure( figsize = (12, 10) )
plt.imshow( plate_corrected, cmap = 'gray', vmin = 0, vmax = 255 );

## Clean up

In [ ]:
del plate_warp
del color_plate_warp

In [ ]:
print( color_plate_corrected.dtype, plate_corrected.dtype )

color_plate_corrected = (255 * color_plate_corrected).astype( np.uint8 )

print( color_plate_corrected.dtype, plate_corrected.dtype )

# Extract boxes with data

The figure we processing has one graph box. 

The graph has a grid, but we will use Gaussian filters to remove it so that we are left with the graph box. 

We then identify its contour based on side.


## Remove grid lines from graph boxes


In [ ]:
zoom_factor = 2
x = 200
y = 400

fig = plt.figure(figsize = (10, 4))
ax1 = fig.add_subplot(121)

grayscale_zoom( ax1, plate_corrected, x, y, zoom_factor )

ax2 = fig.add_subplot(122)

sigma = 3
img2 = gaussian( plate_corrected, sigma = (sigma, sigma), 
                 truncate = 3.5, preserve_range = True )

plate_for_boxes = img2 > threshold_otsu(img2)
print(f"The array plate_for_boxes is of type {plate_for_boxes.dtype}.\n")

grayscale_zoom( ax2, plate_for_boxes, x, y, zoom_factor )

plt.tight_layout()


In [ ]:
fig = plt.figure( figsize = (12, 10) )
plt.imshow(plate_for_boxes, cmap = 'gray');

Pretty cool, don't you think?

## Contours

We can now identify contours and eliminate all that are small.


In [ ]:
contours = find_contours(plate_for_boxes)
print(f"The algorithm found {len(contours)} contours.\n")

for j in range(len(contours)-1, -1, -1):
    if len(contours[j]) < 3000:
        contours.pop(j)

print(f"There are {len(contours)} good contours.\n" )


In [ ]:
fig = plt.figure( figsize = (12, 10) )
ax = fig.add_subplot(111)

ax.imshow(plate_for_boxes, cmap = 'gray')

# Find coordinates of corners of boxes
#
box_max = []
box_min = []
for n, contour in enumerate(contours):
    ax.plot(contour[:, 1], contour[:, 0], linewidth = 2)
    box_max.append( np.max(contour, axis = 0) )
    box_min.append( np.min(contour, axis = 0) )
    
del contours

We now store the sections of the image with the graph box and with the corresponding text.

In [ ]:
graph_box = color_plate_corrected[int(box_min[0][0]):int(box_max[0][0]), 
                                  int(box_min[0][1]):int(box_max[0][1]), :]

text_box = plate_corrected[int(box_min[0][0]):int(box_max[0][0])+50, 
                           :int(box_min[0][1])]

print(f"The array in graph_box is of type {graph_box.dtype}.\n")
print(f"The array in text_box is of type {text_box.dtype}.\n")

In [ ]:
del plate_for_boxes
del plate_corrected
del color_plate_corrected

In [ ]:
fig = plt.figure( figsize = (10, 10) )
gs = fig.add_gridspec(1, 5)
ax = []

ax.append( fig.add_subplot( gs[0, 0] ) )
ax[-1].imshow( text_box, cmap = 'gray', vmin = 0, vmax = 255 )
    
ax.append( fig.add_subplot( gs[0, 1:] ) )
ax[-1].imshow( graph_box )

plt.tight_layout()


## Save graph box

In [ ]:
display_all_channels(graph_box)

In [ ]:
imshow(text_box, cmap = 'gray');

In [ ]:
imsave( results_folder / 'graph_box_very_noisy.png', graph_box )
imsave( results_folder / 'text_box_very_noisy.png', text_box )

# Next lesson

[click here](nb_05_Text_and_data_in_images.ipynb)